![title](img/logo_white_full.png)

# Regression Models Tutorial

In this part of tutorial, we are going to explain **regression machine learning models** used for insurance data problem. The estimators considered:
* **Linear Regression** (linear)
* **Lasso** (linear)
* **Support Vector Regression** (non-linear) 
* **Decision Tree** (non-linear) 
* **Random Forest** (non-linear, bagging)
* **LightGBM Regressor** (non-linear, boosting)
* **Neural Networks** (non-linear, deep learning)

The tutorial is based on [Allstate Claim Severity Kaggle competition data](https://www.kaggle.com/c/allstate-claims-severity). 

In [1]:
import pickle # Load and save Python objects

import numpy as np # Arrays
import pandas as pd # Data-Frames
from plotly.offline import init_notebook_mode # Plotly

from sklearn.linear_model import LinearRegression, Lasso # Linear models
from sklearn.svm import SVR # SVR
from sklearn.tree import DecisionTreeRegressor # Decision Tree
from sklearn.ensemble import RandomForestRegressor # Random Forest
from lightgbm import LGBMRegressor # LightGBM Regressor
from keras.wrappers.scikit_learn import KerasRegressor # Keras Regressor

from sklearn.preprocessing import PolynomialFeatures # Interactions for linear models
from sklearn.metrics import mean_absolute_error, make_scorer # MAE

from utils import plot_nn_history # Custom Utilities written for this tutorial

import warnings # Ignore annoying warnings
warnings.filterwarnings('ignore')

# Required for Jupyter to produce in-line Plotly graphs
init_notebook_mode(connected=True)

Using TensorFlow backend.


In most cases the best performance is reached by LightGBM or Deep Neural Networks (DNN), so why to bother with other estimators? Sometimes, it can turn out that linear models such as Linear Regression or Lasso are almost as good as more sophisticated estimators. Then, it might be more beneficial for the insurer or bank to use simpler models, as they are more interpretable (at least in regulator's point of view). 

It is a good point to make a note here - please remember, no matter what our clients say - it is a data scientist responsibility to teach banks and insurers the advantages of complex machine learning models. There are certain reasons why AI is so successful, and you with a little work you can also make _a black box_ more interpretable with clever visualizations. 

Ok, let's back to the tutorial. We are going to:
1. Download the training and testing **data** produced in the Data Processing Tutorial. Once again, we do not use Kaggle test dataset as it is unlabelled.
2. Show how to **train each estimator** and what is the best practice. 
3. Check the **performance** on the testing data.

---
## Data
Download the training and testing data that we have created during Data Processing Tutorial.

In [2]:
with open('data/data.pkl', 'rb') as f:
    X_train, X_test, y_train, y_test = pickle.load(f)
print(X_train.shape)
print(X_test.shape)

(141238, 130)
(47080, 130)


Furthermore, let's download the names of columns, as we will use them for results analysis.

In [3]:
with open('data/features.pkl', 'rb') as f:
    features = pickle.load(f)
pd.DataFrame(features).transpose()

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,128,129
0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14


---
## A note on MAE
Mean Absolute Error (MAE) is the metric used for evaluation in this Kaggle competition. In Data Processing Tutorial, we log-transformed the losses to produce less skewed distribution. Since MAE should be calculated on untransformed losses, let's define a custom scorer that will be used for performance evaluation.

In [4]:
def mae_from_logs_score(y_true, y_pred):
    return mean_absolute_error(np.exp(y_true), np.exp(y_pred))

mae_from_logs_scorer = make_scorer(mae_from_logs_score, greater_is_better=False)

---
## Linear Regression
![lr](img/lr.png)
Linear Regression (LR) is the simplest machine learning model known from statistics lessons. It is linear in the coefficients: $$y_i = \alpha_0 + \alpha_1 x_{1,i}+\alpha_2 x_{1,i}+\dots+\alpha_P x_{P,i}$$ and the parameters $\alpha_k, k=0,\dots,P$ can be uniqually determined using [least squares method](https://en.wikipedia.org/wiki/Least_squares) (only if a number of observations $N<=P$).

Let's fit the LR using [scikit-learn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). As Python is an object-oriented programming language, each estimator is actually an object with its methods (```fit```, ```predict```) and properties (```coef_```, ```intercept_```). 

Firstly, we initialize estimator.

In [5]:
lr = LinearRegression()

Now, we are going to use ```fit``` method, to train our estimator on training data using least squares method.

In [6]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

After we determined the parameters of LR, we can access them using ```coef_``` and ```intercept_``` properties.

In [7]:
pd.DataFrame(np.append(lr.intercept_, lr.coef_), index=['intercept']+features).transpose()

,intercept,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,6.426105,-0.173112,0.171549,0.150794,-0.035385,0.046122,-0.178747,0.112539,0.043915,-0.007865,...,0.040195,-0.04259,0.523897,0.092302,0.182382,-0.028205,-0.072335,-0.155528,0.280503,0.237703


We are ready to use ```predict``` method on testing data and calculate the score. We can also calculate MAE on training data, but please remember that decision about model performance should be always made on scores calculated on testing data (or use cross-validation). 

For LR, you can also use old-fashioned statistical methods such as adjusted R^2, AIC or BIC calculated **on training data**, but testing errors already provide a better estimate of true model error, as well as use fewer assumptions about underlying model. Testing errors can be also used for all models such as Neural Networks or Random Forest, for which there is no definition of AIC or BIC.

In [8]:
y_pred = lr.predict(X_test)
print('The MAE calculated on testing data = {0:.2f}'.format(mae_from_logs_score(y_test, y_pred)))
y_pred_train = lr.predict(X_train)
print('The MAE calculated on training data = {0:.2f}'.format(mae_from_logs_score(y_train, y_pred_train)))

The MAE calculated on testing data = 1292.90
The MAE calculated on training data = 1287.62


**Summary of Linear Regression**
* linear model,
* parameters determined using least squares method,
* for non-linear data, LR will be characterized by high bias, i.e. the model would not catch all complexities of the underlying data and produce poor scores for both training and testing data,
* we can boost the performance of LR by adding interactions between variables, such as: $$\{x_1x_2\}, \{x_1^3x_4\}, \{x_1x_2x_5\}, \dots$$ but the form and number of these interactions is very difficult to be determined,
* visualization methods: residuals plots, Q-Q plots, the statistical significance of parameters, outliers plots.

---
## Lasso
The lasso method is an extension of linear regression with the inclusion of the regularization term. Lasso automatically performs feature selection in the course of training and enhances the prediction accuracy of the statistical model. It partially solves the problem of selecting important interactions in LR, as it automatically chooses the right ones. 

The objective minimized in lasso has a form: $$F=\sum_{i=1}^{N}\left( y_i-\alpha_0-\sum_{k=1}^P \alpha_j x_{ij}\right)^2 + \gamma \sum_{k=1}^P |\alpha_j|$$ which obviously cannot be optimized using least-squres method, thus algorithms like coordinate descent are used.

Let's fit Lasso to our data using [scikit-learn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html). In contrary to LR, in Lasso we have one hyperparameter ```alpha``` (λ in the objective function above) that needs to be tuned by data scientist. As hyperoptimization is explained in another tutorial, we limit ourselves to an arbitrarly selected value (you can change the value by hand if you wish and see the results).

In [9]:
lasso = Lasso(alpha=0.001, random_state=2019)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print('The MAE calculated on testing data = {0:.2f}'.format(mae_from_logs_score(y_test, y_pred)))

The MAE calculated on testing data = 1292.62


As we can see the MAE using Lasso is quite similar compared to LR. Why is that? Well, we forgot to add interactions to our features, so that we could not really take benefits of Lasso! We will add interactions up to degree 3 using [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) ```PolynomialFeatures``` only for some **categorical variables**.

In [11]:
# Indices of some categorical variables
idx = [i for i, f in enumerate(features) if f.startswith('cat1')]

# Create interactions for both training and testing data
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_train_poly = np.concatenate((X_train, poly.fit_transform(X_train[:,idx])), axis=1)
X_test_poly = np.concatenate((X_test, poly.fit_transform(X_test[:,idx])), axis=1)
print(X_train_poly.shape)
print(X_test_poly.shape)

# Fit and predict
lasso = Lasso(alpha=0.001, random_state=2019)
lasso.fit(X_train_poly, y_train)
y_pred = lasso.predict(X_test_poly)
print('The MAE calculated on testing data with interactions = {0:.2f}'.format(mae_from_logs_score(y_test, y_pred)))

The MAE calculated on testing data with interactions = 1246.95


Well, it's quite better. You can play around with engineering various interactions, but to be honest, more sophisticated machine learning models can handle non-linear data better and faster. 

**Summary of Lasso**
* linear model,
* automatically selects important features,
* bias-variance trade-off can be controlled by the shrinkage parameter ```alpha```. Increase in λ leads to decreased variance but increased bias, thus by proper selection of the shrinkage parameter value we can increase the accuracy of the model,
* still linear in coefficient and in case of a lot of interactions we can run out of memory or make the model unstable,
* visualization methods: residuals plots, Q-Q plots, the statistical significance of parameters, outliers plots.

---
## Support Vector Regression
SVR is the first non-linear algorithm that we are going to try out. It gains the non-linearity through a kernel function such as Radial Basis Function (RBF). For more info look at this [great post](https://medium.com/coinmonks/support-vector-regression-or-svr-8eb3acf6d0ff). 

[SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) has one main hyperparameter which is the type of kernel (RBF, linear, polynomial) and then the following hyperparameters are dependent on the type of kernel function. We will arbitrarily select hyperparameters: RBF kernel and then C, gamma and epsilon.

**Don't bother to run this piece, of code - it is extremely slow.**

In [ ]:
#svr = SVR(kernel='rbf', C=1, gamma='scale', epsilon=.1)
#svr.fit(X_train, y_train)
#y_pred = svr.predict(X_test)
#print('The MAE calculated on testing data = {0:.2f}'.format(mae_from_logs_score(y_test, y_pred)))

**Summary of Support Vector Regression**
* non-linear model,
* very slow in comparison to other non-linear models,
* on perceptual tasks (vision, speech and so on), they are massively outclassed by deep neural networks. On structured data, they are outperformed by gradient boosted machines. 

---
## Decision Tree
Mathematically speaking, decision tree algorithm relies on a stratification of feature space $X_1, X_2, \dots, X_P$ into $K$ non-overlapping regions $R_1, R_2, \dots, R_K$. Based on the training examples, we calculate mean values $y_{R_k}$ in each region $k \in 1,2,\dots,K$ and these become predictions for each new observation falling into a region $R_k$. The training of decision trees involves proper stratification of the feature space in the regions $R_1, R_2, \dots, R_K$ with an aim to minimize a selected metric, like RSS for regression problems.

You can build a great intuition of this algorithm, reading [this post](https://towardsdatascience.com/decision-trees-in-machine-learning-641b9c4e8052).

In [7]:
tree = DecisionTreeRegressor(random_state=2019)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print('The MAE calculated on testing data = {0:.2f}'.format(mae_from_logs_score(y_test, y_pred)))

The MAE calculated on testing data = 1733.64


As you can see, the MAE is much worse than linear models. Decision trees for bigger training sets are characterized by low bias and high variance, which results in an overfitting problem. To deal with that problem we can merge many decision trees to lower variance. Please look at the next model - Random Forest.

**Summary of Decision Tree**
* non-linear model,
* easy to explain - it results in a set of rules,
* low bias, but very high variance, resulting in poor predictions,
* basis of powerful bagging and boosting algorithms,
* visualization methods: residuals plots, tree methods in case feature space is small.

---
## Random Forest
![rf](img/rf.png)
Random forests overcome this flaw by averaging many decision trees based on the different subsets of a training data. The averaged decision trees are called weak learners, whereas random forests estimator is a strong learner. The decrease of variance is achieved by combining uncorrelated weak learners, which is achieved by bootstrap aggregation (**bagging**) on decision trees and **random selection of features subset** in each tree split. To accomplish this task for each $b=1,2,\dots,B$, where $B$ is the number of averaged decision trees, we perform: 
* Sample $n$ observations with replacement from the training set $X$ and $Y$. These become $X_b$ and $Y_b$.
* Train decision tree $\hat{f}_b$ using $X_b$ and $Y_b$. During learning, randomly select feature subset that will be used in each tree split. 

Once training is completed, predicted value for an unseen observation is calculated as **average of individual decision trees** predictions $$\hat{f}=\frac{1}{B}\sum_{b=1}^B \hat{f}_b$$ Averaging $B$ decision trees trained on $B$ bootstrapped training sets **reduces the varianc**e, greatly **increasing the accuracy and stability** of the statistical model in comparison to single decision trees. 

As each individual decision tree is a non-linear model by dividing the feature space into smaller sub-spaces, random forests have an advantage of capturing non-linear effects, in contrary to linear regression or lasso. Moreover, less data preprocessing is required, as random forests can be used without interaction terms, scaling or transforming data.

Let's fit the estimator to our training data and see the performance.

In [5]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=2019)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print('The MAE calculated on testing data = {0:.2f}'.format(mae_from_logs_score(y_test, y_pred)))

The MAE calculated on testing data = 1209.29


It takes a while to train a model, but we greatly improved MAE in comparison to linear models. Furthermore, we can play around with hyperparameters to improve this score even more, but for now, let's see if we can reach improvement with better models.

**Summary of Random Forest**
* non-linear model,
* reduction in overfitting: by averaging several trees, there is a significantly lower risk of overfitting,
* less variance: By using multiple trees, you reduce the chance of stumbling across a classifier that doesn’t perform well because of the relationship between the train and test data,
* visualization methods: residual plots, feature importance, examples of bagged trees decision rules.

---
## LightGBM Regressor
GBM stands for Gradient Boosting Machines, which are greatly explained [here](https://towardsdatascience.com/understanding-gradient-boosting-machines-9be756fe76ab?gi=555cf4533233). LightGBM and XGBoost are excellent implementations of the GBM algorithm and provides a great improvement of accuracy and execution time comparing to scikit-learn AdaBoost. 

The LightGBM library comes with a great scikit-learn wrapper for LightGBM, so that we can still use familiar ```fit```, ```predict```, etc. methods! Let's fit the estimator with default hyperparameters.

In [6]:
gbm = LGBMRegressor(n_estimators=100, n_jobs=-1, random_state=2019)
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_test)
print('The MAE calculated on testing data = {0:.2f}'.format(mae_from_logs_score(y_test, y_pred)))

The MAE calculated on testing data = 1152.14


Well, clearly it is much faster than random forest. It also provides us a good MAE improvement!. To be honest LightGBM or XGBoost are the algorithms that Kaggle's grandmasters always use in their solutions.

**Summary of LightGBM**
* non-linear model
* boosting methods outperform random forest
* much faster and less memory consumption than scikit-learn implementations of GBM
* paralell mode enabled and GPU-ready

---
## Neural Networks
![nn](img/nn.png)
Last, but not least - neural networks.

Artificial neural networks (ANN) are computing systems inspired by biological neural networks located in animal brains and are used to approximate functions that are unknown. The basic component of ANN is an artificial neuron and the way how the neurons are connected is called a network topology. There are various types of ANN including feedforward neural network, radial basis function network (RBF) or recurrent neural network (RNN) with the first being most commonly used in machine learning applications. The fully-connected feedforward neural network consists of several layers including an input layer, a number of hidden layers and one output layer. It is characterized by the fact that each neuron in a layer has only directed connections to all neurons of the next layer. To describe the fully-connected feedforward neural network we use standard notation. The $j$-th neuron in the $l$-th layer of a network takes a number of inputs, $a_k^{(l-1)}$, where $k=1,\dots, n$ is an index of a neuron in the previous layer consisting of $n$ neurons. An input is then multiplied by a corresponding weight, $w_{jk}^{(l)}$ and an output $a_j^{(l)}$ is calculated by some nonlinear function $g(z_j^{(l)})$ of the weighted input
$$z_j^{(l)}=\sum_{k=1}^n w_{jk}^{(l)} a_k^{(l-1)}$$
The function $g$ is known as _activation function_ and the most commonly used functions are Heaviside step function, sigmoid function and Rectified Linear Unit (RELU) function. The output functions in the last layer are restricted by the type of supervised machine learning algorithm. In classification problems, we are interested in outputs of range $[0,1]$, therefore sigmoid function can be used, whereas in regression problems the output function is set as $g(z_j^{(l)})=z_j^{(l)}$. 

To produce the desired output given an input, ANN are trained by a learning algorithm. The learning algorithm optimizes weights with an aim of minimizing a predefined cost function such as mean squared error (MSE) in regression problems. The weights of ANN can be optimized with gradient descent procedures using efficient backpropagation algorithm. Generally, in each iteration of the backpropagation algorithm a gradient descent procedure updates the weights in the direction of the greatest decrease of a cost function:
$$\delta w_{jk}^{(l)} = - \eta \frac{\partial C}{\partial w_{jk}^{(l)}}$$
where $C$ is the cost function and $\eta$ is an adjustable nonnegative constant, which controls the rate of learning.

We use Keras, a great library that is "like" scikit-wrapper for Tensorflow. First, we need to **build an architecture** of our neural network. Here is the great function that returns us the Keras NN model with standard fully-conntected deep learning architecure. Please note that there are some extras:
* **Batch normalization layer** - helps to avoid overfitting by normalizing the neurons outputs.
* **Dropout layer** - helps to avoid overfitting by 'forgetting' neurons outputs at some rate.
As it is the regression problem, the last (output) layer is just a linear function. 

In [5]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout
    
def nn_architecture(n_features, n_layers=3, n_neurons=512, act='relu', drop=0.0,
                    loss='mean_squared_error', optimizer='Adam'):
    
    # Input
    x = Input(name='inputs', shape=(n_features, ), dtype='float32')
    o = x
    
    # Network
    for layer in range(n_layers):
        o = Dense(n_neurons, activation=act, name='dense' + str(layer + 1))(o)
        if drop > 0:
            o = Dropout(drop, name='drop' + str(layer + 1))(o)
        o = BatchNormalization(name='norm' + str(layer + 1))(o)
    o = Dense(1, activation='linear', name='output')(o)
    
    # Compile and return
    model = Model(inputs=x, outputs=o)
    model.summary()
    model.compile(loss=loss, optimizer=optimizer)
    
    return model

Now, we initilize the estimator, with default architecture (3 layers, 512 neurons in each layer, activation function is RELU). The loss optimized is MSE and the optimizer used is Adam. We use mini-batch stochastic gradient descent with batch size of 512 and 10 epochs (in that case 1 epoch means N/batch_size runs of backpropagation algorithm, where N is the size of training data).

In [11]:
nn = KerasRegressor(build_fn=nn_architecture, epochs=10, batch_size=512, n_features=len(features))
fit_history = nn.fit(X_train, y_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 130)               0         
_________________________________________________________________
dense1 (Dense)               (None, 512)               67072     
_________________________________________________________________
norm1 (BatchNormalization)   (None, 512)               2048      
_________________________________________________________________
dense2 (Dense)               (None, 512)               262656    
_________________________________________________________________
norm2 (BatchNormalization)   (None, 512)               2048      
_________________________________________________________________
dense3 (Dense)               (None, 512)               262656    
_________________________________________________________________
norm3 (BatchNormalization)   (None, 512)               2048      
__________

Please note that the MAE loss monitored is calculated on log-losses, that is why it so small. You can define your custom objective in Keras, but it is more advanced material. Now, as we optimized the weights of neural network, we can check the MAE on testing data.

In [12]:
y_pred = nn.predict(X_test)
print('The MAE calculated on testing data = {0:.2f}'.format(mae_from_logs_score(y_test, y_pred)))

The MAE calculated on testing data = 1294.93


Well, the MAE is not so good as we can expect from such a great algorithm. My result is 1294.93, you can have different as Tensorflow randomly initilize weights (and you have no control over it). Learning of neural network is an art, and therfore we need to do more.

Firstly, we need to watch out for overfitting. As we monitor only loss on the training data, we can set ```validation_split``` argument to non-zero, so that some % of training data will be hold just for calculation of validation errors.

As gradient descent reach the local minimum it can miss it due to the large learning rate. Keras has an implementation of automatic learning rate schedule ```ReduceLROnPlateu```, which will decrease the learning rate by ```factor``` after the validation error has not been improved for the last ```patience``` epochs. 

Moreover, we will increase number of epochs to a large value and use ```EarlyStopping``` callback, so that the training will stop if validation error has not been improved for the last ```patience``` epochs. 

In [6]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
callbacks = [EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='min'),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=0.01, mode='min')]

In [7]:
nn = KerasRegressor(build_fn=nn_architecture, epochs=90, batch_size=512, n_features=len(features),
                    validation_split=0.1, callbacks=callbacks)
fit_history = nn.fit(X_train, y_train)
y_pred = nn.predict(X_test)
print('The MAE calculated on testing data = {0:.2f}'.format(mae_from_logs_score(y_test, y_pred)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 130)               0         
_________________________________________________________________
dense1 (Dense)               (None, 512)               67072     
_________________________________________________________________
norm1 (BatchNormalization)   (None, 512)               2048      
_________________________________________________________________
dense2 (Dense)               (None, 512)               262656    
_________________________________________________________________
norm2 (BatchNormalization)   (None, 512)               2048      
_________________________________________________________________
dense3 (Dense)               (None, 512)               262656    
_________________________________________________________________
norm3 (BatchNormalization)   (None, 512)               2048      
__________

Now, it is better. My MAE on testing data is 1198.56 so we are reaching the LightGBM result. As you can see the learning of deep neural networks is not so easy, but once you apply a correct method you can outperform LightGBM or XGBoost results. 

Additionally, we can plot learning history to see the convergence, analyse the bias-variance tradeoff and overfitting.

In [9]:
plot_nn_history(fit_history.history, metric_name='loss', epoch_start=2)

---
## Further notes
* Try to build similar tutorial on classification dataset.
* Add some visualiztion plots in Plotly such as residual plots, feature importance plots for tree methods, etc.
* Check out next tutorials - hyperoptimization and stacking to improve the predictability.
* Play around with neural networks - it is extremely interesting topic.